<a href="https://colab.research.google.com/github/saakshi20/Neural_Network_SMS_Text_Classifier/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the training and test datasets
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convert labels to binary (ham = 0, spam = 1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Separate features and labels
X_train = train_df['message'].values
y_train = train_df['label'].values
X_test = test_df['message'].values
y_test = test_df['label'].values

print("Training data size:", len(X_train))
print("Test data size:", len(X_test))


In [ ]:
# Tokenize the text data
max_vocab = 10000  # Maximum number of words in the vocabulary
max_len = 100       # Maximum length of sequences

# Create tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_vocab, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_padded = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_padded = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

print("Training sequence shape:", X_train_padded.shape)
print("Test sequence shape:", X_test_padded.shape)


In [ ]:
# Build the improved model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_vocab, output_dim=32, input_length=max_len),  # Increased output dimension
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(32, activation='relu'),     # Added another dense layer
    keras.layers.Dropout(0.3),                     # Added dropout for regularization
    keras.layers.Dense(1, activation='sigmoid')    # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Retrain the model
history = model.fit(
    X_train_padded, y_train,
    epochs=15,                # Increased epochs
    batch_size=32,
    validation_data=(X_test_padded, y_test),
    verbose=1
)


In [ ]:
# Train the model
history = model.fit(
    X_train_padded, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test_padded, y_test),
    verbose=1
)


In [ ]:
def predict_message(pred_text):
    # Tokenize and pad the input message
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len, padding='post')

    # Make a prediction
    prediction = float(model.predict(padded_sequence)[0][0])  # Ensure float conversion

    # Set label based on threshold
    label = "spam" if prediction >= 0.5 else "ham"

    # Return the prediction score and label
    return [prediction, label]


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
